# Amazon Nova Act SDK を使用したライブビュー Browser ツール

## 概要

このチュートリアルでは、Nova Act SDK を使用して Amazon Bedrock AgentCore Browser ツールを操作し、ブラウザをライブで表示する方法を学びます。


### チュートリアル詳細


| 項目         | 詳細                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| チュートリアルタイプ       | 会話型                                                                   |
| エージェントタイプ          | シングル                                                                           |
| エージェントフレームワーク   | Nova Act                                                                         |
| LLM モデル           | Amazon Nova Act モデル                                                            |
| チュートリアルコンポーネント | Nova Act を使用したライブでの Browser ツール操作                                 |
| チュートリアル分野   | クロスバーティカル                                                                   |
| 難易度  | 簡単                                                                             |
| 使用 SDK            | Amazon Bedrock AgentCore Python SDK、Nova Act                                     |

### チュートリアルアーキテクチャ

このチュートリアルでは、Nova Act と Browser ツールを使用してブラウザをライブで表示する方法を説明します。  

この例では、Nova Act エージェントに自然言語の指示を送り、Bedrock AgentCore Browser 上でタスクを実行しながらブラウザをライブで表示します。

<div style="text-align:left">
    <img src="./images/browser-tool.png" width="50%"/>
</div>

### チュートリアルの主な機能

* Browser ツールの使用とライブビュー表示 
* Nova Act と Browser ツールの連携

## 前提条件

このチュートリアルを実行するには、以下が必要です：
* Python 3.10+
* AWS 認証情報。IAM ロール/ユーザーには以下の権限が必要です：https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/browser-onboarding.html#browser-credentials-config
* Amazon Bedrock AgentCore SDK
* Nova Act SDK および API キー 

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

## ライブビュー付きで Bedrock AgentCore Browser ツールを Nova Act で使用

ここでは、Amazon DCV SDK 経由で Bedrock AgentCore Browser ツールに接続するためのヘルパー関数を使用します。




In [ ]:
%%writefile live_view_with_nova_act.py
from bedrock_agentcore.tools.browser_client import browser_session
from nova_act import NovaAct
from rich.console import Console
from rich.panel import Panel
import sys
import json
import argparse
sys.path.append("../interactive_tools")
from browser_viewer import BrowserViewerServer

console = Console()

from boto3.session import Session

boto_session = Session()
region = boto_session.region_name
print("使用リージョン", region)

def live_view_with_nova_act(prompt, starting_page, nova_act_key, region="us-west-2"):
    """ディスプレイサイズを設定してブラウザライブビューワーを実行します。"""
    console.print(
        Panel(
            "[bold cyan]ブラウザライブビューワー[/bold cyan]\n\n"
            "このデモで説明する内容：\n"
            "・DCV によるライブブラウザ表示\n"
            "・設定可能なディスプレイサイズ（900x800 に限定されません）\n"
            "・適切なディスプレイレイアウトコールバック\n\n"
            "[yellow]注意：Amazon DCV SDK ファイルが必要です[/yellow]",
            title="ブラウザライブビューワー",
            border_style="blue",
        )
    )

    try:
        # ステップ 1: ブラウザセッションを作成
        with browser_session(region) as client:
            ws_url, headers = client.generate_ws_headers()

            # ステップ 2: ビューワーサーバーを開始
            console.print("\n[cyan]ステップ 3: ビューワーサーバーを開始中...[/cyan]")
            viewer = BrowserViewerServer(client, port=8000)
            viewer_url = viewer.start(open_browser=True)

            # ステップ 3: 機能を表示
            console.print("\n[bold green]ビューワー機能：[/bold green]")
            console.print(
                "・デフォルトディスプレイ：1600x900（displayLayout コールバックで設定）"
            )
            console.print("・サイズオプション：720p、900p、1080p、1440p")
            console.print("・リアルタイムディスプレイ更新")
            console.print("・制御の取得/解放機能")

            console.print("\n[yellow]停止するには Ctrl+C を押してください[/yellow]")

            # ステップ 4: Nova Act を使用してブラウザを操作
            with NovaAct(
                cdp_endpoint_url=ws_url,
                cdp_headers=headers,
                preview={"playwright_actuation": True},
                nova_act_api_key=nova_act_key,
                starting_page=starting_page,
            ) as nova_act:
                result = nova_act.act(prompt)
                console.print(f"\n[bold green]Nova Act 結果：[/bold green] {result}")
        
    except Exception as e:
        console.print(f"\n[red]エラー: {e}[/red]")
        import traceback
        traceback.print_exc()
    finally:
        console.print("\n\n[yellow]シャットダウン中...[/yellow]")
        if "client" in locals():
            client.stop()
            console.print("ブラウザセッションを終了しました")
    return result


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--prompt", required=True, help="ブラウザ検索の指示")
    parser.add_argument("--starting-page", required=True, help="開始 URL")
    parser.add_argument("--nova-act-key", required=True, help="Nova Act API キー")
    parser.add_argument("--region", default="us-west-2", help="AWS リージョン")
    args = parser.parse_args()

    result = live_view_with_nova_act(
        args.prompt, args.starting_page, args.nova_act_key, args.region
    )

    with open('result.txt', 'w') as f:
        f.write(str(result))

    console.print(f"\n[bold green]Nova Act 結果：[/bold green] {result}")

#### スクリプトの実行
スクリプトを実行する前に、以下に Nova Act API キーを貼り付けてください。 

In [ ]:
NOVA_ACT_KEY= ''  ### Paste your Nova Act Key here

In [ ]:
!python live_view_with_nova_act.py --prompt "Search for macbooks and extract the details of the first one" --starting-page "https://www.amazon.com/" --nova-act-key {NOVA_ACT_KEY}

### 裏側で何が起きていたか？ 
* Browser クライアントをインスタンス化してセッションを開始しました
* その後、`BrowserViewerServer` を使用してブラウザセッションに接続し、セッションをローカルで表示しました
* Nova Act Agent を作成し、ブラウザセッションの詳細を渡しました
* その後、Nova Act エージェントに自然言語の指示を送り、アクションをライブで確認しました

## ブラウザでの CAPTCHA の処理 
次に、ブラウザで CAPTCHA を処理する方法を見てみましょう。CAPTCHA の目的は、Web サイトと対話しているのがボットではなく人間であることを確認することです。そのため、エージェントに CAPTCHA を解かせるのではなく、人間が制御を引き継ぎ、CAPTCHA を解決してからエージェントに続行させます。 

新しいスクリプトを作成しましょう。Nova Act はページ上に CAPTCHA があるかどうかを確認できます。この機能を使用して、Nova Act が続行する前に CAPTCHA を処理します。 

#### スクリプトの実行が開始されると、ブラウザのローカルビューが表示されます。スクリプトの実行中に CAPTCHA が表示された場合は、手動で CAPTCHA を解いてください。スクリプトは CAPTCHA が解決されるまで待機します。 

##### 注意：CAPTCHA が表示されずにスクリプトが正常に完了した場合は、CAPTCHA に遭遇するまでスクリプトを再実行してみてください。 

In [ ]:
%%writefile captcha_with_nova_act.py
from bedrock_agentcore.tools.browser_client import browser_session
from nova_act import NovaAct, BOOL_SCHEMA, ActAgentError
from rich.console import Console
from rich.panel import Panel
import sys
import json
import time
import argparse
sys.path.append("../interactive_tools")
from browser_viewer import BrowserViewerServer


console = Console()

from boto3.session import Session

boto_session = Session()
region = boto_session.region_name
print("使用リージョン", region)

def contains_human_validation_error(err):
    """
    エラーまたはその message 属性が HumanValidationError を示しているかを再帰的にチェックします。
    """
    if err is None:
        return False

    # 文字列の直接チェック
    if isinstance(err, str) and "HumanValidationError" in err:
        return True

    # err に文字列または別のエラーの 'message' 属性がある場合、再帰
    if hasattr(err, "message"):
        return contains_human_validation_error(err.message)

    # err の文字列表現にエラーテキストが含まれている場合
    if "HumanValidationError" in str(err):
        return True

    return False

def live_view_with_nova_act(steps, starting_page, nova_act_key, region="us-west-2"):
    """ディスプレイサイズを設定してブラウザライブビューワーを実行します。"""
    console.print(
        Panel(
            "[bold cyan]ブラウザライブビューワー[/bold cyan]\n\n"
            "このデモで説明する内容：\n"
            "・DCV によるライブブラウザ表示\n"
            "・設定可能なディスプレイサイズ（900x800 に限定されません）\n"
            "・適切なディスプレイレイアウトコールバック\n\n"
            "[yellow]注意：Amazon DCV SDK ファイルが必要です[/yellow]",
            title="ブラウザライブビューワー",
            border_style="blue",
        )
    )
    result = None

    try:
        # ステップ 1: ブラウザセッションを作成
        with browser_session(region) as client:
            ws_url, headers = client.generate_ws_headers()

            # ステップ 2: ビューワーサーバーを開始
            console.print("\n[cyan]ステップ 3: ビューワーサーバーを開始中...[/cyan]")
            viewer = BrowserViewerServer(client, port=8000)
            viewer_url = viewer.start(open_browser=True)

            # ステップ 3: 機能を表示
            console.print("\n[bold green]ビューワー機能：[/bold green]")
            console.print(
                "・デフォルトディスプレイ：1600x900（displayLayout コールバックで設定）"
            )
            console.print("・サイズオプション：720p、900p、1080p、1440p")
            console.print("・リアルタイムディスプレイ更新")
            console.print("・制御の取得/解放機能")

            console.print("\n[yellow]停止するには Ctrl+C を押してください[/yellow]")

            # ステップ 4: Nova Act を使用してブラウザを操作
            with NovaAct(
                cdp_endpoint_url=ws_url,
                cdp_headers=headers,
                preview={"playwright_actuation": True},
                nova_act_api_key=nova_act_key,
                starting_page=starting_page,
            ) as nova_act:
                
                for step_index, step in enumerate(steps):
                    max_retries = 3
                    retry_count = 0
                    
                    while retry_count < max_retries:
                        try:
                            print(f"ステップ {step_index + 1}/{len(steps)} を実行中: {step}")
                            result = nova_act.act(step)
                            console.print(f"\n[bold green]ステップ {step_index + 1} 結果：[/bold green] {result}")
                            break  # 成功、次のステップへ
                            
                        except ActAgentError as err:
                            # エラーメッセージまたは構造内のヒューマンバリデーションをチェック
                            if contains_human_validation_error(err):
                                print("CAPTCHA を検出しました！ブラウザで解いてください。")
                                captcha_wait_attempts = 0
                                max_captcha_wait_attempts = 8
                                
                                while captcha_wait_attempts < max_captcha_wait_attempts:
                                    try:
                                        time.sleep(10)  # ユーザーが CAPTCHA を解く時間を与える
                                        captcha_result = nova_act.act(
                                            "Is there a captcha on the screen?", schema=BOOL_SCHEMA
                                        )
                                        
                                        if captcha_result.matches_schema and not captcha_result.parsed_response:
                                            print("CAPTCHA が解決されました。現在のステップを続行します...")
                                            # retry_count をインクリメントしないので、ペナルティなしで現在のステップをリトライ
                                            break
                                        else:
                                            print(f"CAPTCHA がまだ表示されています。待機中... (試行 {captcha_wait_attempts + 1}/{max_captcha_wait_attempts})")
                                            captcha_wait_attempts += 1
                                            
                                    except Exception as captcha_check_err:
                                        print(f"CAPTCHA 状態チェック中にエラー: {str(captcha_check_err)}")
                                        captcha_wait_attempts += 1
                                        time.sleep(5)
                                
                                if captcha_wait_attempts >= max_captcha_wait_attempts:
                                    print("CAPTCHA 待機の最大試行回数に達しました。続行を試みます。")
                                    retry_count += 1
                                
                            else:
                                print(f"CAPTCHA 以外のエラーが発生しました: {str(err)}")
                                retry_count += 1
                                time.sleep(5)
                                
                        except Exception as general_err:
                            print(f"ステップ {step_index + 1} で予期しないエラー: {str(general_err)}")
                            retry_count += 1
                            time.sleep(5)
                            
                    if retry_count >= max_retries:
                        console.print(f"\n[bold red]{max_retries} 回の試行後、ステップ {step_index + 1} を完了できませんでした。[/bold red]")
                        if step_index < len(steps) - 1:
                            console.print("[yellow]次のステップへの続行を試みます...[/yellow]")
                
                # 最終サマリー
                console.print("\n[bold blue]タスク実行完了[/bold blue]")
        
    except Exception as e:
        console.print(f"\n[red]エラー: {e}[/red]")
        import traceback
        traceback.print_exc()
    finally:
        console.print("\n\n[yellow]シャットダウン中...[/yellow]")
        if "client" in locals():
            client.stop()
            console.print("ブラウザセッションを終了しました")
    return result


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--steps", required=False, help="実行するステップの JSON 配列またはカンマ区切りリスト", 
                        default='["Search for AI news and press enter. If there is alredy AI news typed in search bar, then do not do anything", "Get the first AI news result, open the page and extract the title. Instead, if you see an AI summary, extract the first paragrpah of the summary and return"]')
    parser.add_argument("--starting-page", required=True, help="開始 URL")
    parser.add_argument("--nova-act-key", required=True, help="Nova Act API キー")
    parser.add_argument("--region", default="us-west-2", help="AWS リージョン")
    args = parser.parse_args()

    # ステップをパース - JSON 配列またはカンマ区切り値のどちらかを受け付ける
    try:
        # まず JSON としてパースを試みる
        steps = json.loads(args.steps)
    except json.JSONDecodeError:
        # 有効な JSON でない場合、カンマ区切り文字列として扱う
        steps = [step.strip() for step in args.steps.split(',')]

    # steps がリストであることを確認
    if not isinstance(steps, list):
        steps = [steps]

    result = live_view_with_nova_act(
        steps, args.starting_page, args.nova_act_key, args.region
    )

In [ ]:
!python captcha_with_nova_act.py  --starting-page "https://www.google.com/" --nova-act-key {NOVA_ACT_KEY}

# おめでとうございます！